In [3]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
import os
import yaml

# Carregar a chave da API
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Inicializando o modelo OpenAI
openai = ChatOpenAI(model="gpt-4", temperature=0)

# Templates para os diferentes tipos de perguntas
financial_template = PromptTemplate.from_template(
    """
    Você é um especialista financeiro.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Financeiro".
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
)

tech_support_template = PromptTemplate.from_template(
    """
    Você é um especialista em suporte técnico.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
)

# Criando as chains de LLM para cada tipo de pergunta
financial_chain = LLMChain(prompt=financial_template, llm=openai)
tech_support_chain = LLMChain(prompt=tech_support_template, llm=openai)

# Função de roteamento que seleciona a chain correta
def route(info):
    topic = info["topic"].lower()
    
    # Melhorando a detecção de palavras-chave na query
    if "senha" in topic or "técnico" in topic or "resetar" in topic or "redifinir" in topic:
        print("tech \n\n\n\n\n")
        return tech_support_chain  # Se for sobre "senha" ou "problemas técnicos", usa o suporte técnico
    elif "financeiro" in topic:
        print("tech \n\n\n\n\n")
        return financial_chain  # Se for sobre "finanças", usa a cadeia de financeiro
    else:
        print("Genérico \n\n\n\n\n")
        # Retornar uma resposta genérica para outras categorias
        return LLMChain(prompt=PromptTemplate.from_template("Como posso ajudá-lo? Pergunta: {query} Resposta:"), llm=openai)

# Função para invocar o roteamento com a query do usuário
def classify_and_respond(query):
    # Classificação baseada no tópico
    classification = {
        "topic": query  # Aqui você pode usar algum modelo ou lógica para determinar o tópico
    }
    
    # Passa a classificação para a função de roteamento
    selected_chain = route(classification)
    
    # Responde com a chain apropriada
    response = selected_chain.run({"query": query})
    return response

# Testando com uma pergunta
query = "Como faço para redefinir minha senha?"
response = classify_and_respond(query)
print(response)


tech 





Bem-vindo ao Suporte Técnico. Para redefinir sua senha, geralmente você precisa ir à página de login do serviço específico e clicar em "Esqueceu sua senha" ou "Redefinir senha". Em seguida, siga as instruções fornecidas, que normalmente envolvem receber um e-mail com um link para redefinir sua senha. Se você não receber o e-mail, verifique sua pasta de spam. Se você ainda estiver tendo problemas, entre em contato conosco novamente para obter mais assistência.
